In this block of code it will ask for your token from Hugging Face.

If you don't have an account you can make one here: https://huggingface.co/join

Before you can do this you need to accept the license agreement for the model, which you can do here: https://huggingface.co/CompVis/stable-diffusion-v1-4

On the token page select "new token" and ask for a "read" token (you can name it "Colab").

In [ ]:
!pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate networks lora utils pillow
!git clone https://github.com/RandomGamingDev/MCSkinsGen
!huggingface-cli login

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from safetensors.torch import load_file

scheduler = PNDMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True)

#@markdown Stable Diffusion Model
model_id = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}
#@markdown The Device (this is mostly just for if you're running it on a different machine)
device = "cuda" #@param {type:"string"}

pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, revision="fp16", safety_checker=None, use_auth_token=True)
pipe = pipe.to(device)

#@markdown LoRA Model
lora_model = "mcskins-18.safetensors" #@param {type:"string"}

pipe.load_lora_weights(f"/MCSkinsGen/models/{ lora_model }")


Here is where you actually make images. Change the "prompt" to whatever you want to try and then change "num_images" if you want more than one image generated. You can re-run this cell without having to re-run everything before it.

In [ ]:
import PIL

#@markdown Generation Parameters
prompt = "A Alchemist minecraft skin in the tv category. Alchemist with golden hair." #@param {type:"string"}
num_images = 2 #@param {type:"number"}

prompts = [prompt] * num_images

#@markdown AI Parameters
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
#@markdown Recommended values: <br/>
#@markdown Most Recommended: `guidance_scale=7.5, num_inference_steps=50` <br/>
#@markdown `guidance_scale=15, num_inference_steps=50 or 100 or 200` <br/>
#@markdown `guidance_scale=30, num_inference_steps=100`

with autocast("cuda"):
    images = pipe(prompts, guidance_scale=7.5, num_inference_steps=50).images

#@markdown To download the skins simply right click and press `Save As`
images = [image.resize((64, 64), resample=PIL.Image.Resampling.NEAREST) for image in images]
media.show_images(images)